In [1]:
import os
import numpy as np
import matplotlib.pyplot as plt
import torch

from noteify.core.config import SAMPLE_RATE
from noteify.core.datasets import (MusicNetDataset, MusicNetDatasetProcessed, MusicAugmentor,
                                   MusicNetSampler, get_musicnet_dataloader)
from noteify.core.training import make_optimizer, make_scheduler, train_model
from noteify.core.models import TranscriptionNN
from noteify.core.utils import plot_audio, plot_roll_info
from noteify.utils import get_rel_pkg_path

In [2]:
device = torch.device('cuda' if torch.cuda.is_available() else "cpu")
if torch.cuda.is_available():
    print("Using the GPU!")
else:
    print("Warning: Could not find GPU! Using CPU only")

Using the GPU!


In [3]:
data_dir = get_rel_pkg_path("musicnet/")
save_dir = get_rel_pkg_path("weights/")

In [4]:
raw_dataset_train = MusicNetDataset(data_dir, download=True, train=True,
                                    numpy_cache=True, piano_only=True) #, filter_records=[1727])
raw_dataset_test = MusicNetDataset(data_dir, download=True, train=False,
                                   numpy_cache=True, piano_only=True) #, filter_records=[1759])

  2%|█▉                                                                              | 15/640 [00:00<00:04, 133.59it/s]

Loading audio


100%|████████████████████████████████████████████████████████████████████████████████| 20/20 [00:00<00:00, 1571.28it/s]

Loading audio


In [5]:
batch_size = 32

In [6]:
dataset_train = MusicNetDatasetProcessed(raw_dataset_train, augmentor=MusicAugmentor())
sampler_train = MusicNetSampler(dataset_train, batch_size, random_start_times=True)
workers = 3 if raw_dataset_train.piano_only else 2
dataloader_train = get_musicnet_dataloader(dataset_train, sampler_train, num_workers=workers, pin_memory=True)

dataset_test = MusicNetDatasetProcessed(raw_dataset_test)
sampler_test = MusicNetSampler(dataset_test, batch_size)
dataloader_test = get_musicnet_dataloader(dataset_test, sampler_test, num_workers=None, pin_memory=True)

dataloaders = {'train': dataloader_train, 'test': dataloader_test}

In [7]:
model = TranscriptionNN()
model = model.to(device)
#past_train_weights = os.path.abspath(os.path.join(save_dir, "Experiment 12-07-2020 08-29AM/Weights Best.pckl"))
#model.load_state_dict(torch.load(past_train_weights))

CQT kernels created, time used = 0.0947 seconds


In [8]:
optimizer = make_optimizer(model, lr=0.001)

In [9]:
num_epochs = 50
scheduler = make_scheduler(optimizer, [60], gamma=0.5)

In [ ]:
tracker = train_model(device=device,
                      model=model,
                      dataloaders=dataloaders,
                      optimizer=optimizer,
                      lr_scheduler=scheduler,
                      save_model=True,
                      save_dir=save_dir,
                      save_best=True,
                      save_all=False,
                      num_epochs=num_epochs)

  0%|                                                                                          | 0/309 [00:00<?, ?it/s]

Epoch 0/49
----------


Avg. Loss: 0.0579, Total Loss: 0.0060, Loss Parts: [0.0025, 0.0017, 0.0018]: 100%|███| 309/309 [06:01<00:00,  1.17s/it]


Training Loss: 0.0579
Test statistics: {'frame_avg_precision': 0.12496272332665831, 'reg_onset_mae': 0.34845757484436035, 'reg_offset_mae': 0.260353147983551}


  0%|                                                                                          | 0/309 [00:00<?, ?it/s]

...

Avg. Loss: 0.0211, Total Loss: 0.0049, Loss Parts: [0.0022, 0.0014, 0.0014]: 100%|███| 309/309 [06:12<00:00,  1.20s/it]


Training Loss: 0.0211
Test statistics: {'frame_avg_precision': 0.34990552210971926, 'reg_onset_mae': 0.37136542797088623, 'reg_offset_mae': 0.28732916712760925}


  0%|                                                                                          | 0/309 [00:00<?, ?it/s]


Epoch 2/49
----------


Avg. Loss: 0.0215, Total Loss: 0.0063, Loss Parts: [0.0028, 0.0017, 0.0018]:  97%|██▉| 299/309 [05:57<00:06,  1.54it/s]

In [ ]:
%qtconsole